In [10]:
import boto3

In [11]:
session = boto3.Session()
s3 = session.resource('s3')
bucket = s3.Bucket("andrew-triton-bucket")

In [6]:
for item in bucket.objects.all():
    print(item.key)

model.graphdef


# Example with Deployer

In [1]:
import wandb
import os
import tensorflow as tf

In [2]:
config = {
    "entity": "megatruong",
    "project": "fashion-mnist-keras-triton",
    "artifact_name": "model-sage-feather-1",
    "artifact_version": 1,
    "triton_url": "localhost:8000",
}

assert isinstance(
    config["artifact_version"], int
), "Triton requires model version to be an integer"
assert "triton_url" in config, "Triton URL must be specified in config"

run = wandb.init(config=config, job_type='deploy_to_triton')

wandb: Currently logged in as: megatruong. Use `wandb login --relogin` to force relogin


In [3]:
import shutil

def copy_to_new_path(old_path, new_path):
    """Copy a directory to a new path, creating the new path if it doesn't exist."""
    os.makedirs(new_path, exist_ok=True)
    shutil.copytree(old_path, new_path, dirs_exist_ok=True)


def make_triton_compatible_name(name):
    """Convert wandb artifact name to Triton-compatible name"""
    return name.replace("-", "_")

In [4]:
wandb_artifact_str = (
    "{entity}/{project}/{artifact_name}:v{artifact_version}".format(**run.config)
)

In [5]:
wandb_artifact_str

'megatruong/fashion-mnist-keras-triton/model-sage-feather-1:v1'

In [6]:
art = run.use_artifact(wandb_artifact_str)
artifact_path = art.download()

wandb: Downloading large artifact model-sage-feather-1:v1, 273.69MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.1


In [7]:
artifact_path

'./artifacts/model-sage-feather-1:v1'

In [8]:
model = tf.keras.models.load_model(artifact_path)
model_name = make_triton_compatible_name(run.config["artifact_name"])
tf.keras.models.save_model(model, os.path.join("./models", model_name, str(run.config['artifact_version']), "model.savedmodel"))

INFO:tensorflow:Assets written to: ./models/model_sage_feather_1/1/model.savedmodel/assets


INFO:tensorflow:Assets written to: ./models/model_sage_feather_1/1/model.savedmodel/assets


In [21]:
for root, dir, files in os.walk('./models'):
    for f in files:
        print(root)

./models/model_sage_feather_1/1/model.savedmodel
./models/model_sage_feather_1/1/model.savedmodel
./models/model_sage_feather_1/1/model.savedmodel/variables
./models/model_sage_feather_1/1/model.savedmodel/variables


In [ ]:
s3_client = boto3.client('s3')

In [27]:
path = 'models'

for root, _, files in os.walk(path):
    for f in files:
        full_path = os.path.join(root, f)
        print(full_path)
        s3_client.upload_file(full_path, 'andrew-triton-bucket', full_path)
        # full_path = os.path.join(root, f)
        # rel_path = os.path.relpath(full_path, path)
        # remote_obj_path = os.path.join(remote_path, rel_path)
        # print(f"Uploading {rel_path} to {remote_obj_path}")
        # s3_client.upload_file(full_path, config["triton_bucket"], remote_obj_path)

models/model_sage_feather_1/1/model.savedmodel/keras_metadata.pb
models/model_sage_feather_1/1/model.savedmodel/saved_model.pb
models/model_sage_feather_1/1/model.savedmodel/variables/variables.data-00000-of-00001
models/model_sage_feather_1/1/model.savedmodel/variables/variables.index


In [13]:
uploadDirectory('./models', 'andrew-triton-bucket')

AttributeError: 's3.ServiceResource' object has no attribute 'upload_file'

In [14]:
import s3fs

ModuleNotFoundError: No module named 's3fs'

# Interact with Tritonserver

In [12]:
import tritonclient.http as httpclient

In [13]:
client = httpclient.InferenceServerClient(url="localhost:8000", verbose=True)

In [29]:
from rich import print

In [40]:
config = client.get_model_config("model-sage-feather-1")

GET /v2/models/model-sage-feather-1/config, headers None
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '1116'}>
bytearray(b'{"name":"model-sage-feather-1","platform":"tensorflow_savedmodel","backend":"tensorflow","version_policy":{"specific":{"versions":[1]}},"max_batch_size":8,"input":[{"name":"input_1","data_type":"TYPE_FP32","format":"FORMAT_NONE","dims":[32,32,3],"is_shape_tensor":false,"allow_ragged_batch":false,"optional":false}],"output":[{"name":"conv5_block3_out","data_type":"TYPE_FP32","dims":[1,1,2048],"label_filename":"","is_shape_tensor":false}],"batch_input":[],"batch_output":[],"optimization":{"priority":"PRIORITY_DEFAULT","input_pinned_memory":{"enable":true},"output_pinned_memory":{"enable":true},"gather_kernel_buffer_threshold":0,"eager_batching":false},"dynamic_batching":{"preferred_batch_size":[8],"max_queue_delay_microseconds":0,"preserve_ordering":false,"priority_levels":0,"default_priority_level":0,"priority_que

In [44]:
client.get_model_repository_index()

POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '302'}>
bytearray(b'[{"name":"model-sage-feather-1","version":"1","state":"READY"},{"name":"model-sage-feather-1_copy_0","version":"1","state":"READY"},{"name":"model-sage-feather-1_copy_1","version":"1","state":"READY"},{"name":"model-sage-feather-1_copy_2","version":"1","state":"READY"},{"name":"model_sage_feather_1"}]')


[{'name': 'model-sage-feather-1', 'version': '1', 'state': 'READY'},
 {'name': 'model-sage-feather-1_copy_0', 'version': '1', 'state': 'READY'},
 {'name': 'model-sage-feather-1_copy_1', 'version': '1', 'state': 'READY'},
 {'name': 'model-sage-feather-1_copy_2', 'version': '1', 'state': 'READY'},
 {'name': 'model_sage_feather_1'}]

In [35]:
from google.protobuf.json_format import ParseDict

In [33]:
ParseDict(config)

AttributeError: 'dict' object has no attribute 'DESCRIPTOR'

In [31]:
print(config)

{
    'name': 'model_sage_feather_1',
    'platform': 'tensorflow_savedmodel',
    'backend': 'tensorflow',
    'version_policy': {'latest': {'num_versions': 1}},
    'max_batch_size': 8,
    'input': [
        {
            'name': 'input_1',
            'data_type': 'TYPE_FP32',
            'format': 'FORMAT_NONE',
            'dims': [32, 32, 3],
            'is_shape_tensor': False,
            'allow_ragged_batch': False,
            'optional': False
        }
    ],
    'output': [
        {
            'name': 'conv5_block3_out',
            'data_type': 'TYPE_FP32',
            'dims': [1, 1, 2048],
            'label_filename': '',
            'is_shape_tensor': False
        }
    ],
    'batch_input': [],
    'batch_output': [],
    'optimization': {
        'priority': 'PRIORITY_DEFAULT',
        'input_pinned_memory': {'enable': True},
        'output_pinned_memory': {'enable': True},
        'gather_kernel_buffer_threshold': 0,
        'eager_batching': False
    },
    'dynamic_batching': {
        'preferred_batch_size': [8],
        'max_queue_delay_microseconds': 0,
        'preserve_ordering': False,
        'priority_levels': 0,
        'default_priority_level': 0,
        'priority_queue_policy': {}
    },
    'instance_group': [
        {
            'name': 'model_sage_feather_1',
            'kind': 'KIND_CPU',
            'count': 2,
            'gpus': [],
            'secondary_devices': [],
            'profile': [],
            'passive': False,
            'host_policy': ''
        }
    ],
    'default_model_filename': 'model.savedmodel',
    'cc_model_filenames': {},
    'metric_tags': {},
    'parameters': {},
    'model_warmup': []
}

In [18]:
config.update(max_batch_size=8)

In [22]:
import json

In [23]:
client.load_model('model_sage_feather_1', config=json.dumps(config))

POST /v2/repository/models/model_sage_feather_1/load, headers None
{"parameters":{"config":"{\"name\": \"model_sage_feather_1\", \"platform\": \"tensorflow_savedmodel\", \"backend\": \"tensorflow\", \"version_policy\": {\"latest\": {\"num_versions\": 1}}, \"max_batch_size\": 8, \"input\": [{\"name\": \"input_1\", \"data_type\": \"TYPE_FP32\", \"format\": \"FORMAT_NONE\", \"dims\": [32, 32, 3], \"is_shape_tensor\": false, \"allow_ragged_batch\": false, \"optional\": false}], \"output\": [{\"name\": \"conv5_block3_out\", \"data_type\": \"TYPE_FP32\", \"dims\": [1, 1, 2048], \"label_filename\": \"\", \"is_shape_tensor\": false}], \"batch_input\": [], \"batch_output\": [], \"optimization\": {\"priority\": \"PRIORITY_DEFAULT\", \"input_pinned_memory\": {\"enable\": true}, \"output_pinned_memory\": {\"enable\": true}, \"gather_kernel_buffer_threshold\": 0, \"eager_batching\": false}, \"dynamic_batching\": {\"preferred_batch_size\": [4], \"max_queue_delay_microseconds\": 0, \"preserve_orderin

In [ ]:
def deploy_n_times(model, n):
    client.load_model('')

In [1]:
# test async

In [2]:
import tritonclient.http.aio as asyncclient

In [4]:
client = asyncclient.InferenceServerClient(url="localhost:8000", verbose=True)

In [9]:
await client.load_model('model-sage-feather-1_copy_0')
await client.load_model('model-sage-feather-1_copy_1')
await client.load_model('model-sage-feather-1_copy_2')

POST http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_0/load, headers None
{}
<ClientResponse(http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_0/load) [200 OK]>
<CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '0')>

Loaded model 'model-sage-feather-1_copy_0'
POST http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_1/load, headers None
{}
<ClientResponse(http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_1/load) [200 OK]>
<CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '0')>

Loaded model 'model-sage-feather-1_copy_1'
POST http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_2/load, headers None
{}
<ClientResponse(http://localhost:8000/v2/repository/models/model-sage-feather-1_copy_2/load) [200 OK]>
<CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '0')>

Loaded model 'model-sage-feather-1_copy_2'


In [10]:
jobs = [f'model-sage-feather-1_copy_{i}' for i in range(3)]

In [16]:
def do_work(j):
    return client.load_model(j)

In [21]:
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as exc:
    exc.map(do_work, jobs)

POST /v2/repository/models/model-sage-feather-1_copy_0/load, headers None
{}
POST /v2/repository/models/model-sage-feather-1_copy_1/load, headers None
{}
POST /v2/repository/models/model-sage-feather-1_copy_2/load, headers None
{}


KeyboardInterrupt: 

In [20]:
for j in jobs:
    do_work(j)

POST /v2/repository/models/model-sage-feather-1_copy_0/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'model-sage-feather-1_copy_0'
POST /v2/repository/models/model-sage-feather-1_copy_1/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'model-sage-feather-1_copy_1'
POST /v2/repository/models/model-sage-feather-1_copy_2/load, headers None
{}
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '0'}>
Loaded model 'model-sage-feather-1_copy_2'


In [26]:
config = client.get_model_config("model-sage-feather-1_copy_0")

GET /v2/models/model-sage-feather-1_copy_0/config, headers None
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '1130'}>
bytearray(b'{"name":"model-sage-feather-1_copy_0","platform":"tensorflow_savedmodel","backend":"tensorflow","version_policy":{"specific":{"versions":[1]}},"max_batch_size":8,"input":[{"name":"input_1","data_type":"TYPE_FP32","format":"FORMAT_NONE","dims":[32,32,3],"is_shape_tensor":false,"allow_ragged_batch":false,"optional":false}],"output":[{"name":"conv5_block3_out","data_type":"TYPE_FP32","dims":[1,1,2048],"label_filename":"","is_shape_tensor":false}],"batch_input":[],"batch_output":[],"optimization":{"priority":"PRIORITY_DEFAULT","input_pinned_memory":{"enable":true},"output_pinned_memory":{"enable":true},"gather_kernel_buffer_threshold":0,"eager_batching":false},"dynamic_batching":{"preferred_batch_size":[8],"max_queue_delay_microseconds":0,"preserve_ordering":false,"priority_levels":0,"default_priority_level":0

In [33]:
from google.protobuf.json_format import Parse, ParseDict
from google.protobuf import any_pb2

In [35]:
from rich import print

In [63]:
configs = [client.get_model_config(f"model-sage-feather-1_copy_{i}") for i in range(1)]
bs = [c.get('max_batch_size') for c in configs]

bs

GET /v2/models/model-sage-feather-1_copy_0/config, headers None
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '1132'}>
bytearray(b'{"name":"model-sage-feather-1_copy_0","platform":"tensorflow_savedmodel","backend":"tensorflow","version_policy":{"latest":{"num_versions":1}},"max_batch_size":16,"input":[{"name":"input_1","data_type":"TYPE_FP32","format":"FORMAT_NONE","dims":[32,32,3],"is_shape_tensor":false,"allow_ragged_batch":false,"optional":false}],"output":[{"name":"conv5_block3_out","data_type":"TYPE_FP32","dims":[1,1,2048],"label_filename":"","is_shape_tensor":false}],"batch_input":[],"batch_output":[],"optimization":{"priority":"PRIORITY_DEFAULT","input_pinned_memory":{"enable":true},"output_pinned_memory":{"enable":true},"gather_kernel_buffer_threshold":0,"eager_batching":false},"dynamic_batching":{"preferred_batch_size":[16],"max_queue_delay_microseconds":0,"preserve_ordering":false,"priority_levels":0,"default_priority_level"

[16]

In [64]:
print(configs[0])

{
    'name': 'model-sage-feather-1_copy_0',
    'platform': 'tensorflow_savedmodel',
    'backend': 'tensorflow',
    'version_policy': {'latest': {'num_versions': 1}},
    'max_batch_size': 16,
    'input': [
        {
            'name': 'input_1',
            'data_type': 'TYPE_FP32',
            'format': 'FORMAT_NONE',
            'dims': [32, 32, 3],
            'is_shape_tensor': False,
            'allow_ragged_batch': False,
            'optional': False
        }
    ],
    'output': [
        {
            'name': 'conv5_block3_out',
            'data_type': 'TYPE_FP32',
            'dims': [1, 1, 2048],
            'label_filename': '',
            'is_shape_tensor': False
        }
    ],
    'batch_input': [],
    'batch_output': [],
    'optimization': {
        'priority': 'PRIORITY_DEFAULT',
        'input_pinned_memory': {'enable': True},
        'output_pinned_memory': {'enable': True},
        'gather_kernel_buffer_threshold': 0,
        'eager_batching': False
    },
    'dynamic_batching': {
        'preferred_batch_size': [16],
        'max_queue_delay_microseconds': 0,
        'preserve_ordering': False,
        'priority_levels': 0,
        'default_priority_level': 0,
        'priority_queue_policy': {}
    },
    'instance_group': [
        {
            'name': 'model-sage-feather-1_copy_0',
            'kind': 'KIND_CPU',
            'count': 2,
            'gpus': [],
            'secondary_devices': [],
            'profile': [],
            'passive': False,
            'host_policy': ''
        }
    ],
    'default_model_filename': 'model.savedmodel',
    'cc_model_filenames': {},
    'metric_tags': {},
    'parameters': {},
    'model_warmup': []
}

In [54]:
for c in configs:
    print(c.get('max_batch_size'))

4

In [56]:
from google.protobuf.json_format import MessageToDict

In [62]:
from tensorflow.core.framework import config_pb2 

ImportError: cannot import name 'config_pb2' from 'tensorflow.core.framework' (/home/codespace/.python/current/lib/python3.10/site-packages/tensorflow/core/framework/__init__.py)

Bad pipe message: %s [b'\xb2V\x86$\xf5\x99\x92\x00U\xdcUU\xd9\xac\x08\xe0\x06^ \xb34\x0b{\xc2Ql\xe9\xd8\xc8\xae~\x88\x85|\xaa3\x8e\xe3\xaf\xa87\x88\x8a>Y.\xb5', b'\xdcJ\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00']
Bad pipe message: %s [b'\x157\x95\xe3\xed)\xea\x83T|~\x13*\x08\x00\x97a\x16 \xbe\x95\x10{\xd8u \xb9\x9b\xeb;\xe1\x9bf\t"\xee\xf5{\x01k\\\xb4 \xc1\xdd\xfb\xc51\xb6\xcb\xac\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08', b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'']
Bad pipe message: %s [b"0\xbd'Q\xf33\xf2\xa38\xd8\x0bEk\xd6V<\x0c \x00

In [60]:
from tensorflow.core.framework import graph_pb2 as gpb

gdef = gpb.GraphDef()

with open('config.pbtxt') as f:
    text_format.Parse(f.read(), gdef)

ParseError: 1:1 : Message type "tensorflow.GraphDef" has no field named "name".

In [59]:
MessageToDict('config.pbtxt')

AttributeError: 'str' object has no attribute 'DESCRIPTOR'